# Source 
- [Blog Yaae: Automatic Differentiation Explanation](https://hackmd.io/@machine-learning/blog-post-yaae#I-Introduction)
- [Youtube: L6.2 Understanding Automatic Differentiation via Computation Graphs](https://www.youtube.com/watch?v=oY6-i2Ybin4)

In [1]:
import numpy as np

In [2]:
class CustomNumber:
    def __init__(self, num):
        self.num = num 

    def __add__(self, other):
        return self.num + other 

    def __sub__(self, other):
        return self.num - other
    

In [3]:
a = CustomNumber(2)

In [4]:
a - 1

1

In [5]:
x = np.array([1, 2])
x

array([1, 2])

In [6]:
y = np.array([1,2,3])
y

array([1, 2, 3])

In [7]:
np.outer(x, y).T

array([[1, 2],
       [2, 4],
       [3, 6]])

In [8]:
class Test:
    def angka(self, x):
        return 1

    def angka(self, x, y):
        return 2

In [9]:
test = Test()

In [10]:
test.angka(1, 1)

2

In [11]:
def topo_sort(v, L, visited):
        """
            Returns list of all of the children in the graph in topological order.
            Parameters:
            - v: last node in the graph.
            - L: list of all of the children in the graph in topological order (empty at the beginning).
            - visited: set of visited children.
        """
        if v not in visited:
            visited.add(v)
            for child in v.children:
                topo_sort(child, L, visited)
            L.append(v)
        return L

In [12]:
def compress_gradient(grad, other_tensor_shape):
    """
        Returns the gradient but compressed (needed when gradient shape mismatch during reverse mode).

        Paramaters:
        - grad: gradient.
        - other_tensor_shape: shape of target tensor.
    """
    ndims_added = grad.ndim - len(other_tensor_shape)
    for _ in range(ndims_added): 
        grad = grad.sum(axis=0)         
    for i, dim in enumerate(other_tensor_shape):
        if dim == 1: 
            grad = grad.sum(axis=i, keepdims=True) 
    return grad

In [13]:
class Node:
    def __init__(self, data, requires_grad=True, children=[]):

        self.data = data if isinstance(data, np.ndarray) else np.array(data)
        self.requires_grad = requires_grad
        self.children = children 
        self.grad= None 

        if self.requires_grad:
            self.zero_grad()

        self._compute_derivatives = lambda: None

    @property
    def shape(self):
        return self.data.shape

    def zero_grad(self):
        self.grad = Node(np.zeros_like(self.data, dtype=np.float64), requires_grad=False)

    def backward(self, grad=None):
        L, visited = [], set()
        topo = topo_sort(self, L, visited)

        if grad is None:
            if self.shape == ():
                self.grad = Node(1, requires_grad=False)
            else:
                raise RuntimeError('backward: grad must be specified for non-0 tensor')
        else:
            self.grad = Node(grad, requires_grad=False) if isinstance(grad, (np.ndarray, list)) else grad
    
        for v in reversed(topo):
            v._compute_derivatives()

    # operators
    def __add__(self, other):
        op = Add(self, other)
        out = op.forward_pass()
        out._compute_derivatives = op.compute_derivatives 
        return out

    def matmul(self, other):
        op = Matmul(self, other)
        out = op.forward_pass()
        out._compute_derivatives = op.compute_derivatives 
        return out

    def sin(self):
        op = Sin(self)
        out = op.forward_pass()
        out._compute_derivatives = op.compute_derivatives 
        return out 

    def relu(self):
        op = ReLU(self)
        out = op.forward_pass()
        out._compute_derivatives = op.compute_derivatives
        return out


In [14]:
class Add():
    
    def __init__(self, node1, node2):
        self.node1 = node1 if isinstance(node1, Node) else Node(node1)
        self.node2 = node2 if isinstance(node2, Node) else Node(node2)
    
    def forward_pass(self):
        self.out = Node(np.add(self.node1.data, self.node2.data), children=[self.node1, self.node2])
        return self.out

    def compute_derivatives(self):
        if self.node1.requires_grad:
            self.node1.grad.data += compress_gradient(self.out.grad.data, self.node1.shape)
        if self.node2.requires_grad:
            self.node2.grad.data += compress_gradient(self.out.grad.data, self.node2.shape)
            


In [15]:
class Matmul():

    def __init__(self, node1, node2):
        self.node1 = node1 if isinstance(node1, Node) else Node(node1)
        self.node2 = node2 if isinstance(node2, Node) else Node(node2)

    def forward_pass(self):
        self.out = Node(self.node1.data @ self.node2.data, children=[self.node1, self.node2])
        return self.out

    def compute_derivatives(self):
        dim = [i for i in range(len(self.node1.data.shape))]
        if len(dim) > 1:
            dim[-1], dim[-2] = dim[-2], dim[-1]

        if self.node1.requires_grad:
            self.node1.grad.data = self.out.grad.data @ self.node2.data.transpose(dim)
        if self.node2.requires_grad:
            self.node2.grad.data = self.node1.data.transpose(dim) @ self.out.grad.data


In [16]:
class Sin():

    def __init__(self, node1):
        self.node1 = node1 if isinstance(node1, Node) else Node(node1)
    
    def forward_pass(self):
        self.out = Node(np.sin(self.node1.data), children=[self.node1])
        return self.out

    def compute_derivatives(self):
        self.node1.grad.data += np.cos(self.node1.data) * self.out.grad.data

In [17]:
class ReLU():

    def __init__(self, node1):
        self.node1 = node1 if isinstance(node1, Node) else Node(node1)

    def forward_pass(self):
        self.out = Node(self.node1.data * (self.node1.data > 0), children=[self.node1])
        return self.out

    def compute_derivatives(self):
        self.node1.grad.data += self.out.grad.data * (self.node1.data > 0)

# Test

In [21]:
w1 = Node(np.array(10), requires_grad=True)
w2 = Node(np.array(10), requires_grad=True)
# w2 = Node([10, 20], requires_grad=True)
w3 = w2 + w1
# w4 = w3.relu()
# w5 = w3 + w4
# z = w5
w3.backward()
print(w1.grad.data) 
# print(w2.grad.data)

1.0
